In [ ]:
import os
import sys 
import gc
import glob
import random 
import cv2
import numpy as np 
import pandas as pd 
from sklearn import metrics
import torch
from torch import nn
import torch.nn.functional as F
from tqdm import tqdm_notebook as tqdm
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

gc.enable()

In [ ]:
BATCH_SIZE = 48 
NUM_FOLDS = 5
NUM_EPOCHS = 3
DEVICE = 'cuda'
LEARNING_RATE = 1e-3

submission_df = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')

In [ ]:
package_path = "../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master/"
sys.path.append(package_path)

import efficientnet_pytorch

In [ ]:
def load_dicom(path):
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array
    data = data - np.min(data)
    if np.max(data) != 0:
        data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data

In [ ]:
class Dataset:
    def __init__(self, paths, targets=None, inference_only=False):
        self.paths = paths
        self.targets = targets
        self.inference_only = inference_only
    
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, index):
        _id = self.paths[index]
        patient_path = f"../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/{str(_id).zfill(5)}/"
        channels = []
        for t in ("FLAIR", "T1w", "T1wCE"): # "T2w"
            t_paths = sorted(
                glob.glob(os.path.join(patient_path, t, "*")), 
                key=lambda x: int(x[:-4].split("-")[-1]),
            )
            # start, end = int(len(t_paths) * 0.475), int(len(t_paths) * 0.525)
            x = len(t_paths)
            if x < 10:
                r = range(x)
            else:
                d = x // 10
                r = range(d, x - d, d)
                
            channel = []
            # for i in range(start, end + 1):
            for i in r:
                channel.append(cv2.resize(load_dicom(t_paths[i]), (256, 256)) / 255)
            channel = np.mean(channel, axis=0)
            channels.append(channel)
        
        if self.inference_only:
            return {
                'X': torch.tensor(channels).float()
            }
        
        return {
            "X": torch.tensor(channels).float(), 
            "y": torch.tensor(self.targets[index], dtype=torch.float),
        }

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = efficientnet_pytorch.EfficientNet.from_name("efficientnet-b0")
        checkpoint = torch.load("../input/efficientnet-pytorch/efficientnet-b0-08094119.pth")
        self.net.load_state_dict(checkpoint)
        n_features = self.net._fc.in_features
        self.net._fc = nn.Linear(in_features=n_features, out_features=1, bias=True)
    
    def forward(self, x):
        out = self.net(x)
        return out

In [ ]:
def predict(model, data_loader):
    model.eval()

    result = np.zeros(len(data_loader.dataset))    
    index = 0
    
    final_predictions = []
    with torch.no_grad():
        for data in data_loader:
            features = data['X']
            features = features.to(DEVICE, dtype=torch.float)
            
            predictions = model(features).squeeze()
            predictions = torch.sigmoid(predictions).cpu().detach().numpy().tolist()
            final_predictions.extend(predictions)
            
        
        return final_predictions

In [ ]:
model_paths = [
    '../input/rsna-brain-train-baseline/model_0.pth',
    '../input/rsna-brain-train-baseline/model_1.pth',
    '../input/rsna-brain-train-baseline/model_2.pth',
    '../input/rsna-brain-train-baseline/model_3.pth',
    '../input/rsna-brain-train-baseline/model_4.pth',
]

test_dataset = Dataset(
    submission_df.BraTS21ID.values,
    inference_only=True
)
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE
)  

In [ ]:
all_predictions = np.zeros((NUM_FOLDS, len(submission_df)))

for i, path in enumerate(model_paths):
    print(f'using {path}...')
    
    model = Model()
    model.load_state_dict(torch.load(path))
    model.to(DEVICE)
    
    all_predictions[i] = predict(model, test_loader)
    
    del model
    gc.collect()

sub_preds = all_predictions.mean(axis=0)

In [ ]:
submission_df['MGMT_value'] = sub_preds
submission_df

In [ ]:
submission_df.to_csv('submission.csv', index=False)